<H1><center>BigQuery Datalab Notebook</center></H1>

The goal of this notebook is to iterate how BigQuery can be currently used in Datalab and list pros and cons of each approach.

We will be running a simple query to find the hight weight of new born twins in the public natality table in BigQuery:

In [78]:
PROJECT_ID = 'bigquery-test-project-166321'
QUERY = "SELECT mother_age,is_male,weight_pounds FROM [bigquery-public-data:samples.natality] WHERE plurality = 2 ORDER BY weight_pounds DESC, mother_age limit 10"

<H2> BigQuery's Official Client<H2>

In [38]:
from google.cloud import bigquery


In [46]:
client = bigquery.Client(project=PROJECT_ID)

query = client.run_sync_query(QUERY)
query.timeout_ms = 10000
query.run()

assert query.complete
rows = query.rows
token = query.page_token

while True:
    print rows
    if token is None:
        break
    rows, total_count, token = query.fetch_data(page_token=token)       # API request

[(23, 18.0007436923), (30, 18.0007436923), (28, 18.0007436923), (28, 18.0007436923), (33, 18.0007436923), (27, 18.0007436923), (18, 18.0007436923), (28, 18.0007436923), (25, 18.0007436923), (22, 18.0007436923)]


<b>- Pros:</b>
  Familiar to BigQuery users (?)
  
<b>- Cons:</b>
  No Datalab/Pandas integration.
  Probably harder to fork and change (and maintain).


<H2>Datalab's BigQuery Module</H2>

To get Pandas integration, we could use Datalab's BigQuery module.

In [80]:
import google.datalab.bigquery as bq

# The QUERY listed in the first cell does not work in datalab for some formatting reason.
QUERY_FOR_DATALAB = "SELECT mother_age,is_male,weight_pounds FROM publicdata.samples.natality WHERE plurality = 2 ORDER BY weight_pounds DESC, mother_age limit 10"
df = bq.Query(QUERY_FOR_DATALAB).execute().result().to_dataframe()
df

,mother_age,is_male,weight_pounds
0,23,True,17.769258
1,19,False,17.636981
2,21,True,17.636981
3,27,False,17.562024
4,21,True,17.416519
5,16,False,17.352585
6,31,False,17.306288
7,26,True,17.145350
8,26,True,17.145350
9,16,False,16.977799


This is how the same thing can be done using magics

In [101]:
%%bigquery execute -q QUERY

mother_age,is_male,weight_pounds
23,True,17.7692583172
19,False,17.63698096
21,True,17.63698096
27,False,17.5620237909
21,True,17.416518698
16,False,17.352584642
31,False,17.306287567
26,True,17.1453501157
26,True,17.1453501157
16,False,16.9777987966


<b>- Pros:</b>
  Magic support.
  
<b>- Cons:</b>
  Limited to Datalab.
  
<b> How to implement: </b> The best way to customize this library is to fork the code in https://github.com/googledatalab/pydatalab and add our own features (for ML). Then ask users to 'pip install' the new package which will automatically replace the default version of this package in Datalab.

<H2>Pandas Bigquery Module</H2>

Finally, we could directly use BigQuery module from Pandas.

In [98]:
import pandas as pd
 
# to communicate with Google BigQuery
from pandas.io import gbq

try:
    top10_baby_df = gbq.read_gbq(QUERY, project_id=PROJECT_ID)
except:
    print 'Error reading the dataset'
    
top10_baby_df

Requesting query... ok.
Query running...
Query done.
Cache hit.

Retrieving results...
Got 10 rows.

Total time taken 0.75 s.
Finished at 2017-05-05 23:10:59.


,mother_age,is_male,weight_pounds
0,23,True,17.769258
1,19,False,17.636981
2,21,True,17.636981
3,27,False,17.562024
4,21,True,17.416519
5,16,False,17.352585
6,31,False,17.306288
7,26,True,17.145350
8,26,True,17.145350
9,16,False,16.977799


<b>- Pros:</b>
  Having a wrapper around this library (or forking the library) will give us maximum flexibility, since people who use the library outside of bq will also have access to the new features.
  
<b>- Cons:</b>
  No support for BigQuery magics.
  
<b> How to implement: </b> We can customize in the same way as we discussed for the Datalab's Bigquery module. The code that we need to fork is https://github.com/pydata/pandas-gbq

<H2> Summary </H2>

Given the discussions above, I think using BigQuery client is probably is not the best idea. We should decide between direct Pandas or Datalab integration. I personally vote for Pandas integration.